In [10]:
import numpy as np
import pandas as pd

In [ ]:
# import weather dataset and select relevant columns
weather_2018 = pd.read_csv('data-raw/weather_2018.csv', index_col=False)
weather_dec_2017 = pd.read_csv('data-raw/weather_dec_2017.csv', index_col=False)
weather = pd.concat([weather_dec_2017, weather_2018])
weather = weather[['DATE', 'HourlyAltimeterSetting', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 
                   'HourlySeaLevelPressure', 'HourlyStationPressure', 'HourlyPressureChange', 'HourlyPressureTendency', 
                   'HourlyVisibility', 'HourlyWindSpeed', 'HourlySkyConditions', 'HourlyPrecipitation', 
                   'HourlyWetBulbTemperature', 'HourlyWindDirection', 'HourlyRelativeHumidity']]


# handle non-numeric values: change those values to NaN and forward fill later
weather.HourlyStationPressure = pd.to_numeric(weather.HourlyStationPressure, errors='coerce')
weather.HourlyVisibility = pd.to_numeric(weather.HourlyVisibility, errors='coerce')
weather.HourlyPrecipitation = pd.to_numeric(weather.HourlyPrecipitation, errors='coerce')
weather.HourlyWindDirection = pd.to_numeric(weather.HourlyWindDirection, errors='coerce')
weather.HourlyAltimeterSetting = pd.to_numeric(weather.HourlyAltimeterSetting, errors='coerce')
weather.HourlySeaLevelPressure = pd.to_numeric(weather.HourlySeaLevelPressure, errors='coerce')

# convert na values to 0 (no pressure change)
weather.HourlyPressureChange = np.nan_to_num(weather.HourlyPressureChange)
weather.HourlyPressureTendency = np.nan_to_num(weather.HourlyPressureTendency)

# transform sky conditions to two dummy columns: cloud_1 == 1 for overcast sky, cloud_0 == 1 for clear sky
weather['cloud_1'] = 0
weather['cloud_0'] = 1
weather.HourlySkyConditions = weather.HourlySkyConditions.fillna(method='ffill')
for i in range(0, len(weather)):
    v = weather.HourlySkyConditions[i]
    if ':08 ' in v or ':09 ' in v or ':10 ' in v:
        weather.cloud_1[i] = 1
        weather.cloud_0[i] = 0


# transform wind direction to four dummie columns: wind_north, wind_east, wind_south, wind_west
weather['wind_north'] = 0
weather['wind_east'] = 0
weather['wind_south'] = 0
weather['wind_west'] = 0
weather.HourlyWindDirection = weather.HourlyWindDirection.fillna(method='ffill')
for i in range(0, len(weather)):
    d = weather.HourlyWindDirection[i]
    if d > 315 or d <= 45:
        weather.wind_north[i] = 1
    elif d > 225 and d <= 315:
        weather.wind_west[i] = 1
    elif d >135 and d <= 225:
        weather.wind_south[i] = 1
    else:
        weather.wind_east[i] = 1

# forward fill the other columns, assume weather doesn't change in an hour
weather = weather.fillna(method='ffill')

C:\Users\bower\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (34,51,76,89) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\bower\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bower\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


pandas.core.frame.DataFrame